# Двоичное дерево и вывод его на экран
Заведем тип для представления двоичных деревьев и вместе напишем код, который будет выводить его на экран:

In [5]:
-- используемые символы: ┌└ ┤│ (возможно, ─)
-- 
--      ┌123
--   ┌56┤
--   │  └78
-- 10┤
--   └42

In [6]:
data BinaryTree = Leaf Int | InnerNode Int BinaryTree BinaryTree

treeExample = InnerNode 10 (InnerNode 56 (Leaf 123) (Leaf 78)) (Leaf 42)

In [35]:
showBinaryTree :: BinaryTree -> String
-- showBinaryTree tree = helperDrawer "" "" "" tree where
-- showBinaryTree = helperDrawer "" "" "" where
-- showBinaryTree tree = helperDrawer "" "" "" tree ++ "\n" where
showBinaryTree = (++ "\n") . helperDrawer "" "" "" where
    helperDrawer _ middlePrefix _ (Leaf value) = middlePrefix ++ show value
    helperDrawer leftPrefix middlePrefix rightPrefix (InnerNode value left right) =
                            helperDrawer (leftPrefix ++ wSpaces ++ " ") -- рисуем левое поддерево
                                         (leftPrefix ++ wSpaces ++ "┌")
                                         (leftPrefix ++ wSpaces ++ "│")
                                         left
                            ++ "\n" ++
                            middlePrefix ++ show value ++ "┤" -- рисуем среднюю (корневую часть дерева)
                            ++ "\n" ++
                            helperDrawer (rightPrefix ++ wSpaces ++ "│") -- рисуем правое поддерево
                                         (rightPrefix ++ wSpaces ++ "└")
                                         (rightPrefix ++ wSpaces ++ " ")
                                         right
                           where
                            wSpaces = replicate (length (show value)) ' '
                         
putStr $ showBinaryTree treeExample

     ┌123
  ┌56┤
  │  └78
10┤
  └42

# Ленивые вычисления
Начнем со встроенного значения `undefined`, при его вычислении всегда возникает ошибка: `undefined = undefined`.



In [36]:
undefined

: 

In [37]:
:type undefined

undefined :: forall a. a

`undefined :: a`, т.е. оно имеет любой тип, его можно вставлять как часть любого другого выражения:

In [40]:
2 + undefined -- например, можно использовать как число

: 

Важная особенность Haskell, ленивые вычисления, это означает, что значения выражений вычисляются только если и только тогда, когда они понадобились:

In [41]:
if 2 > 3 then undefined else 42

42

Ошибки нет, потому что значение не понадобилось!

In [43]:
:type const

const :: forall a b. a -> b -> a

In [45]:
const 10 20
const 10 undefined

Line 1: Evaluate
Found:
const 10 20
Why not:
10Line 2: Evaluate
Found:
const 10 undefined
Why not:
10

10

10

Haskell вычисляет от общего к частному. `if 2 > 3 then undefined else 2 + 2`, сначала вычисляется `if`, получается `2 + 2`, потом вычисляется 2 + 2.

In [55]:
f :: Int -> Int -> Int
f 0 _ = 0
f x y = x * y

f 0 10
f 0 undefined
f 1 undefined

0

0

: 

Как заставить Haskell все-таки вычислить выражение перед использованием. Выражение вычисляется в следующих случаях:
 - происходит операция ввода вывода, например, печать
 - при сопоставлении с образцом
 - при применении функции к аргументу операцией `$!` (не обычный `$`)
 - в алгебраических типах, если аргумент описан с помощью ! (см. далее)
 
Пример вычисления при сопоставлении с образцом:

In [60]:
f :: Int -> Int -> Int
f 0 _ = 0
f _ 0 = 0
f x y = x * y

f 0 undefined -- ok, выполняется первое правило
-- f undefined 0 -- ошибка, уже при попытке проверить первое правило 0 сопоставляется с undefined

f 0 $ undefined -- эквивалентно f 0 undefined
f 0 $! undefined -- ошибка, потребовали вычислить значение перед применением

Line 9: Redundant $
Found:
f 0 $ undefined
Why not:
f 0 undefined

0

0

: 

In [75]:
-- восклицательный знак говорит, что значение надо вычислить
data Pair = Pair Int !Int deriving Show
p1 = Pair 2 3
p2 = Pair undefined 3
p3 = Pair 2 undefined

f :: Pair -> Int
f (Pair _ _) = 0

f p1 -- нет ошибки, p1 хорошая
f p2 -- нет ошибки, p2 содержит undefined там, где можно
f p3 -- ошибка

: 

## Ленивые списки

Введем список из бесконечного числа 1:

In [77]:
list = 1 : list

take 10 list

[1,1,1,1,1,1,1,1,1,1]

list не вычисляется сразу, он хранится так, что чтобы вычислить его голову, надо взять 1, чтобы вычислить хвост, надо взять его же.

In [79]:
take1 :: Int -> [a] -> [a]
take1 0 _ = []
take1 n (h:t) = h : take1 (n-1) t

```
take 2 list = сопоставляем take 2 (1:list) = 1 : take 1 list = 1 : сопоставляем take 1 (1:list) = 1 : 1 : take 0 list = 1 : 1 : [] = [1, 1]
```

In [2]:
list = 1:2:list
take 10 list

list = map (*10) (1:2:list)
take 10 list

zipWith (+) [10, 20, 30] [1, 2, 3] -- объединяет элементы списка указанной операцией

list = 1 : zipWith (+) list list
take 10 list

Line 9: Redundant bracket
Found:
(list)
Why not:
list

[1,2,1,2,1,2,1,2,1,2]

[10,20,100,200,1000,2000,10000,20000,100000,200000]

[11,22,33]

[1,2,4,8,16,32,64,128,256,512]